**A percepção dos usuários de redes sociais sobre a tecnologia de NFT e suas aplicações analisadas sob a ótica da análise de sentimentos com processamento de linguagem natural**

PARTE 1: Coleta de tweets relacionados ao tema NFT

A metodologia de coleta das postagens do Twiter foi realizada com o auxílio da biblioteca Tweepy que fornece recursos para o acesso à API do Twiter. Neste estudo optou-se por analisar as postagens coletadas ao longo de 30 dias consecutivos, no período compreendido entre os dias 09 de fevereiro e 10 de março de 2022. A cada dia do período citado foram coletados 2000 tweets. De cada tweet fora armazenado apenas a informação de data e o texto publicado, sendo desconsideradas todas as demais informações passíveis de coleta pela API. O script em Python desenvolvido para a coleta desconsiderou o registro de tweets repetidos e retweets para evitar que uma grande quantidade de anúncios e spams fizesse parte da composição da amostra. Ao final do período foram armazenados 60 mil registros em duas colunas “data” e “texto/tweet”. 

## Bibliotecas, importações e acessos à API do Twiter

In [ ]:
import pandas as pd
import tweepy as tw
from time import sleep
import datetime as dt
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 2.6 MB/s 


In [ ]:
# Chaves de accesso à API
consumer_key = '###################'
consumer_secret = '###################'
access_token = '###################'
access_token_secret =  '###################'

In [ ]:
# Permissões de acesso
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Coleta de 2 termos

**O Melhor código até o momento**


**Trata a limitação de 900 tt a cada 15 min**

(Como: Com o parâmetro wait_on_rate_limit quando atingir o limite o código aguardará 15 min e seguirá. Obs: permitido apenas tweets de até 7 dias a partir da data de coleta)

**Melhor desempenho em laços**

(Como: O parâmetro retry_count = 5 ajuda a evitar que o código dê erro após fazer mais de 1h rodando (mais de 3600 itens coletados))

**Laço para colher todos os tweets da lista de termos definida:**

(Como: Separa em listas de acordo com o item da query pela posição do item da lista de search do Cursor, permitindo assim várias pesquisas)

**Remove tweets repetidos do dataset de coleta:**

(Como: vai comparar o tweet (30% da segunda metade do texto do tweet) com os outros inclusos em uma string com todos os textos únicos anteriores (vai juntar todos os textos e consultar com find()), se o texto não estiver entre os textos anteriores vai incluí-lo no dataset principal, do contrário irá para o dataset de repetidos)

**Permite definir a quantidade de tweets ÚNICOS coletados**
(Como: O cursor está definido para pegar todos os tweets sobre o tema, mas há um contador no laço for que nos permite coletar uma quantidade específica de tweets únicos, ignorando os repetidos)


### O código

In [ ]:
# Definindo os termos para a consulta
cripto = ["nft","metaverse"]

# Parâmetros e listas de controle para o funcionamento do código
search_words = cripto
new_search = []

desde = "2022-02-10"
ate =  "2022-02-15"

#desde = dt.datetime(year=2022, month=2, day=2, hour=00, minute=00, second=00)
#ate = dt.datetime(year=2022, month=2, day=3, hour=00, minute=00, second=00)


ct,ct1 = "", ""
tweets_bitcoin, tweets_ethereum =  [], []
repetidos1, repetidos2 = [], []

# Laço para agregar o parâmetro de remoção de retweets ao termo da query
for i in range(len(search_words)):
  new_search.append(search_words[i] + " -filter:retweets")

# Laço para fazer o cursor pesquisar cada termo definido pela variável "new_search"
for i in range(len(new_search)):
  cursores = tw.Cursor(api.search, q = new_search[i], lang = 'en', 
                       since = desde, until = ate, wait_on_rate_limit = True, 
                       retry_count =5, retry_delay = 5).items()

# Laço para colher os tweets e facilitar a filtragem nos outros laços
  for tweet in cursores:
    texto_tweet = api.get_status(tweet.id, tweet_mode="extended").full_text.lower()
    ajustado = texto_tweet.replace('\n','').replace(' ','')
    texto_parcial = ajustado[round(len(ajustado) * 0.5) : round(len(ajustado) * 0.8)]

    if i == 0:        # Filtra o termo pela posição
      if ct.find(texto_parcial) == -1:   # Valida se o tweet não é repetido e o registra
        ct += " " + ajustado
        lista2 = [tweet.created_at , texto_tweet]
        tweets_bitcoin.append(lista2)
        if len(tweets_bitcoin) == 1000:   # Quebra o laço quando a coleta alcançar x tweets
          break
      else:
        repetidos1.append(texto_tweet) # Registra os tweets repetidos em outra lista

# Repetição do laço anterior para o termo de pesquisa seguinte:
    if i == 1:
      if ct1.find(texto_parcial) == -1:
        ct1 += " " + ajustado
        lista3 = [tweet.created_at , texto_tweet]
        tweets_ethereum.append(lista3)
        if len(tweets_ethereum) == 1000:
          break
      else:
        repetidos2.append(texto_tweet)

# Dados sobre os tweets repetidos encontrados
print(f'Sobre o tema {cripto[0].upper()} foram encontrados e removidos {len(repetidos1)} tweets repetidos')
print(f'Sobre o tema {cripto[1].upper()} foram encontrados e removidos {len(repetidos2)} tweets repetidos')

# Datasets gerados a partir das coletas (termo1, termo2, repetidos1, repetidos2)
df_bit = pd.DataFrame(tweets_bitcoin, columns= ['data','texto'])
df_ether = pd.DataFrame(tweets_ethereum, columns= ['data','texto'])
repetidos1 = pd.DataFrame(repetidos1)
repetidos2 = pd.DataFrame(repetidos2)

Rate limit reached. Sleeping for: 616
Rate limit reached. Sleeping for: 715
Rate limit reached. Sleeping for: 717


Sobre o tema NFT foram encontrados e removidos 313 tweets repetidos
Sobre o tema METAVERSE foram encontrados e removidos 456 tweets repetidos


In [ ]:
# Todos os tweets ÚNICOS sobre o termo Bitcoin
df_bit

,data,texto
0,2022-02-14 23:59:59,@gordanm00re @uchihai81019569 @soulz_nft @ball...
1,2022-02-14 23:59:58,this is so cool! #nft https://t.co/xc27mk6l0z
2,2022-02-14 23:59:58,"@changerart we want to live quietly, day by da..."
3,2022-02-14 23:59:58,we are giving out og roles to the first 250 pe...
4,2022-02-14 23:59:57,@snoopdogg go check out @generativedng 🍀 this ...
...,...,...
995,2022-02-14 23:54:10,#nft #crypto #cryptocurrency this #valentinesd...
996,2022-02-14 23:54:10,@dinomontez @0xapes_nft noice. 👌🏻
997,2022-02-14 23:54:09,@inmyfeelings321 @0xapes_nft sad to see you go...
998,2022-02-14 23:54:08,👉 check out this excellent nft piece.\n\n 👉 “w...


In [ ]:
# Todos os tweets REPETIDOS sobre o termo Bitcoin
repetidos1

,0
0,@nftparticle hand draw nft https://t.co/sfp7qa...
1,@alienfrens go check out @generativedng 🍀 this...
2,@changerart hand draw nft https://t.co/sfp7qa4wah
3,@newscronos @coingecko @cryptocom @coinbase ht...
4,@kenjorinft polygoon #174\nhttps://t.co/f8xfig...
...,...
308,@smartbeast_nft 🌹 happy valentines day 🌹
309,@kevin_cazun check my new tweet bruh. free nft...
310,@nftrated jasna polska gif 👁\n\ngive offer ‼️\...
311,@thetitanborn have a look at @cyborg86nft \n#5...


In [ ]:
# Todos os tweets ÚNICOS sobre o termo Ethereum
df_ether

,data,texto
0,2022-02-14 23:59:57,fw 2022 ioc nft metaverse collection by \nco-c...
1,2022-02-14 23:59:56,watch: meta's superbowl ad depressingly promot...
2,2022-02-14 23:59:48,@diedaexploring @ajacquitaylor @movingonupnow ...
3,2022-02-14 23:59:45,🦈🦈🦈check it out for the #sharkrace new article...
4,2022-02-14 23:59:43,@thorshammergems $collar will soon take part i...
...,...,...
995,2022-02-14 22:57:20,beta 1.1 is bringing tons of new features into...
996,2022-02-14 22:57:19,gm ans don’t forget the valentine ´ s day part...
997,2022-02-14 22:57:13,@atheistsquid @cernovich well my crypto gains ...
998,2022-02-14 22:57:11,first mma fight in the metaverse || khabib nur...


In [ ]:
# Todos os tweets REPETIDOS sobre o termo Ethereum
repetidos2

,0
0,@afcajax @dt10_official @tobythegoatnft join t...
1,@afcajax @tobythegoatnft join this lovely comm...
2,@afcajax @rgravenberch @tobythegoatnft join th...
3,@afcajax @tobythegoatnft join this lovely comm...
4,@archfiendnft virtual reality soon within ever...
5,@drmaria815 @1inch we don't buy bitcoin \n we ...
6,@hirtzog @behhzad2 @1inch we don't buy bitcoin...
7,@behhzad2 @1inch we don't buy bitcoin \n we ea...
8,@shirley1314a @mikel70490628 we don't buy bitc...
9,@mikel70490628 @1inch we don't buy bitcoin \n ...


### Exportando os dados em arquivo excell (.xlsx)

In [ ]:
writer= pd.ExcelWriter('tweets_criptos_todos.xlsx', engine= 'xlsxwriter')

In [ ]:
df_bit.to_excel(writer, sheet_name='Bitcoin')
df_ether.to_excel(writer, sheet_name='Ethereum')

writer.save()

## Coleta de apenas um termo de pesquisa

In [ ]:
# Coleta de apenas um termo de pesquisa

termo = ["nft"]

# Parâmetros e listas de controle para o funcionamento do código
search_words = termo
ct = ""
new_search, tweets_nft, repetidos =  [], [], []


# data desejada de se obter
desde = "2022-03-12"
# data do dia seguinte
ate =  "2022-03-13"


# Laço para agregar o parâmetro de remoção de retweets ao termo da query
for i in range(len(search_words)):
  new_search.append(search_words[i] + " -filter:retweets")

# Laço para fazer o cursor pesquisar cada termo definido pela variável "new_search"
for i in range(len(new_search)):
  cursores = tw.Cursor(api.search, q = new_search[i], lang = 'en', 
                       since = desde, until = ate, wait_on_rate_limit = True, 
                       retry_count =5, retry_delay = 5).items()

# Laço para colher os tweets e facilitar a filtragem nos outros laços
  for tweet in cursores:
    texto_tweet = api.get_status(tweet.id, tweet_mode="extended").full_text.lower()
    ajustado = texto_tweet.replace('\n','').replace(' ','')
    texto_parcial = ajustado[round(len(ajustado) * 0.5) : round(len(ajustado) * 0.8)]

    if ct.find(texto_parcial) == -1:   # Valida se o tweet não é repetido e o registra
      ct += " " + ajustado
      lista = [tweet.id, tweet.created_at , texto_tweet]
      tweets_nft.append(lista)
      if len(tweets_nft) == 2000:   # Quebra o laço quando a coleta alcançar x tweets
        break
    else:
      repetidos.append(lista) # Registra os tweets repetidos em outra lista

# Datasets gerados a partir das coletas (termo1, termo2, repetidos1, repetidos2)
df_nft = pd.DataFrame(tweets_nft, columns= ['id_tweet','data','texto'])
repetidos = pd.DataFrame(repetidos, columns= ['id_tweet','data','texto'])

# Dados sobre os tweets repetidos encontrados
print(f'Sobre o tema {termo[0].upper()} foram coletados {len(df_nft)} tweets únicos e removidos {len(repetidos)} tweets repetidos')

Rate limit reached. Sleeping for: 785
Rate limit reached. Sleeping for: 789
Rate limit reached. Sleeping for: 786


Sobre o tema OPENSEA foram coletados 2000 tweets únicos e removidos 1055 tweets repetidos


In [ ]:
df_nft.tail()

,id_tweet,data,texto
1995,1502781406203760640,2022-03-12 22:59:42,check out my nft listing on opensea! https://t...
1996,1502781400679866368,2022-03-12 22:59:40,"@frolic if consumption happens from a dapp, yo..."
1997,1502781381675728896,2022-03-12 22:59:36,abstract arts of left side face #nfts #abstrac...
1998,1502781378659819520,2022-03-12 22:59:35,you may purchase my tetra yuga # 19 nft on rar...
1999,1502781376705167361,2022-03-12 22:59:35,check out this item on opensea https://t.co/xg...


In [ ]:
repetidos.tail()

,id_tweet,data,texto
1050,1502781465616072715,2022-03-12 22:59:56,@kenjorijr thank you so much for your space.\n...
1051,1502781451519115270,2022-03-12 22:59:53,check out this item on opensea https://t.co/r5...
1052,1502781406203760640,2022-03-12 22:59:42,check out my nft listing on opensea! https://t...
1053,1502781400679866368,2022-03-12 22:59:40,"@frolic if consumption happens from a dapp, yo..."
1054,1502781400679866368,2022-03-12 22:59:40,"@frolic if consumption happens from a dapp, yo..."


### Exportando dados em excel

In [ ]:
writer= pd.ExcelWriter('tweets_nft.xlsx', engine= 'xlsxwriter')

In [ ]:
df_nft.to_excel(writer, sheet_name='nft')
repetidos.to_excel(writer, sheet_name='repetidos')
writer.save()